In [ ]:
%load_ext autoreload
%autoreload 2

In [40]:
from nn.nn import NeuralNetwork
from nn import io, preprocess 
import numpy as np
import pandas as pd 
import random
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

First, we'll read in the positive and negative sequences. We'll trim the sequences so that they're all the same length (length of the shortest sequence given). Since the positive class is much smaller than the negative class, we'll randomly sample with -----. I chose this method as to not --------. This way, we can preserve all information from the negative sequences. 


In [ ]:
#read in sequence files and combine 
pos_seqs=io.read_text_file('./data/rap1-lieb-positives.txt')
neg_seqs=io.read_fasta_file('./data/yeast-upstream-1k-negative.fa')
all_seqs=pos_seqs+neg_seqs

#get min length
min_seq_len=min(len(min(pos_seqs)), len(min(neg_seqs)))

#trim sequences so that they're all the same length 
trimmed_all_seqs=[]
for seq in all_seqs:
    if len(seq)>min_seq_len:
        start=random.randint(0, len(seq)-min_seq_len)
        trimmed_all_seqs.append(seq[start:start+min_seq_len])
    else:
        trimmed_all_seqs.append(seq)
        
labels=(['positive'] * len(pos_seqs)) + (['negativve'] * len(neg_seqs))        
        

In [21]:
tf_sampled_seqs, tf_sampled_labels=preprocess.sample_seqs(trimmed_all_seqs, labels)
print(len(tf_sampled_seqs))
print(len(tf_sampled_labels))

6326
6326


We'll then one hot encode the sequences, and split into train and test sets. 

In [22]:
#then, one hot encode the sequesnces
one_hot_encode_tf=preprocess.one_hot_encode_seqs(tf_sampled_seqs)

In [37]:
X_train, X_val, y_train, y_val=train_test_split(one_hot_encode_tf, np.expand_dims(tf_sampled_labels, 1), 
                                                  train_size=0.8, random_state=3)
X_train.shape

(5060, 68)

In [38]:
tf_classifier_NN = NeuralNetwork(nn_arch = [{'input_dim': 68, 'output_dim': 34, 'activation': 'relu'},
                                      {'input_dim': 34, 'output_dim': 17, 'activation': 'relu'},
                                      {'input_dim': 17, 'output_dim': 1, 'activation': 'relu'}],
                            lr = 0.0005, seed = 3, batch_size = 100, epochs = 500, loss_function='bce')

In [39]:
per_epoch_loss_train, per_epoch_loss_val=tf_classifier_NN.fit(X_train, y_train, X_val, y_val)

epoch:  0


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U9'), dtype('float64')) -> None

Finally, we'll plot the training and validation loss. 

In [ ]:
#taken from hw7

fig, axs = plt.subplots(2, figsize=(8, 8))
axs[0].plot(np.arange(len(per_epoch_loss_train)), per_epoch_loss_train)
axs[0].set_title('Training Loss History')
axs[1].plot(np.arange(len(per_epoch_loss_val)), per_epoch_loss_val)
axs[1].set_title('Validation Loss History')
plt.xlabel('Steps')
fig.tight_layout()
plt.show()